In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from emnist import extract_training_samples
from sklearn.utils import shuffle
from functools import partial
import time

from vertices_generator import vertices
from kernel import Gaussian_kernel
from mdwsvm import mdwsvm
from mdwsvm_ad import mdwsvm_ad
from one_class_svm import one_class_svm
from hybrid import hybrid
from metric import within_class_error

# Loading Training, Validation, and Test Data

In [ ]:
# Load data
digits_images, digits_labels = extract_training_samples('digits')
letters_images, letters_labels = extract_training_samples('byclass')

# Get number 
mask_1 = (digits_labels == 1)
digits_images_1 = digits_images[mask_1]
digits_labels_1 = digits_labels[mask_1]

mask_3 = (digits_labels == 3)
digits_images_3 = digits_images[mask_3]
digits_labels_3 = digits_labels[mask_3]

mask_5 = (digits_labels == 5)
digits_images_5 = digits_images[mask_5]
digits_labels_5 = digits_labels[mask_5]

mask_7 = (digits_labels == 7)
digits_images_7 = digits_images[mask_7]
digits_labels_7 = digits_labels[mask_7]


# Get letter u, v, w, x, y, z
mask_uvwxyz = (letters_labels == 56) | (letters_labels == 57) | (letters_labels == 58) | (letters_labels == 59) | (letters_labels == 60) | (letters_labels == 61)
letters_images = letters_images[mask_uvwxyz]
letters_labels = letters_labels[mask_uvwxyz]
print(len(letters_labels))

In [ ]:
# Get training and testing data
X_train = np.zeros((800,28,28))
y_train = np.zeros((800), dtype=int)
X_val = np.zeros((8000,28,28))
y_val = np.zeros((8000), dtype=int)
X_test = np.zeros((8000,28,28))
y_test = np.zeros((8000), dtype=int)

# 800 digits normalized training data 
X_train[0:150,:,:] = digits_images_1[0:150,:,:] / 255
X_train[150:300,:,:] = digits_images_3[0:150,:,:] / 255
X_train[300:550,:,:] = digits_images_5[0:250,:,:] / 255
X_train[550:800,:,:] = digits_images_7[0:250,:,:] / 255
X_train = X_train.reshape(800,784).T 
# 800 digits training label
y_train[0:150] = digits_labels_1[0:150]-1
y_train[150:300] = digits_labels_3[0:150]-2
y_train[300:550] = digits_labels_5[0:250]-3
y_train[550:800] = digits_labels_7[0:250]-4

# Get 400 digits for validation X
X_val[0:100,:,:] = digits_images_1[1000:1100,:,:] / 255
X_val[100:200,:,:] = digits_images_3[1000:1100,:,:] / 255
X_val[200:300,:,:] = digits_images_5[1000:1100,:,:] / 255
X_val[300:400,:,:] = digits_images_7[1000:1100,:,:] / 255
# 400 digits validation label
y_val[0:100] = digits_labels_1[1000:1100]
y_val[100:200] = digits_labels_3[1000:1100]
y_val[200:300] = digits_labels_5[1000:1100]
y_val[300:400] = digits_labels_7[1000:1100]
# Get 7600 lowercase letters
X_val[400:8000,:,:] = letters_images[0:7600,:,:] / 255
y_val[400:8000] = letters_labels[0:7600]
# 400 digits and 7600 letters normalized data
X_val = X_val.reshape(8000,784).T

# Get 400 digits for test X
X_test[0:100,:,:] = digits_images_1[1100:1200,:,:] / 255
X_test[100:200,:,:] = digits_images_3[1100:1200,:,:] / 255
X_test[200:300,:,:] = digits_images_5[1100:1200,:,:] / 255
X_test[300:400,:,:] = digits_images_7[1100:1200,:,:] / 255
# 400 digits test label
y_test[0:100] = digits_labels_1[1100:1200]
y_test[100:200] = digits_labels_3[1100:1200]
y_test[200:300] = digits_labels_5[1100:1200]
y_test[300:400] = digits_labels_7[1100:1200]
# Get 7600 lowercase letters
X_test[400:8000,:,:] = letters_images[8000:15600,:,:] / 255
y_test[400:8000] = letters_labels[8000:15600]
# 400 digits and 7600 letters normalized data
X_test = X_test.reshape(8000,784).T

In [ ]:
np.unique(y_train, return_counts=True)

In [ ]:
np.unique(y_val, return_counts=True)

In [ ]:
np.unique(y_test, return_counts=True)

In [ ]:
X_train.shape

Error metric: use metric.py

# MDWSVM

## Cross Validation for MDWSVM

Only use the training data to do k-fold cross validation, and choose the best hyperparameter C based on average cross validation error (sum of all cross validation errors for one C divided by k). Use alpha = 0.5.

In [ ]:
c_values = [2**i for i in range(-3,13)]
w1 = vertices(4)
size = X_train.shape[1]
num_folds = 5

X_train_new, y_train_new = shuffle(X_train.T, y_train, random_state=42)
X_train_new = X_train_new.T
folder_size = int(size / num_folds)
# Loop over each value of c and perform cross-validation
best_c = 0
best_score = -1
for c in c_values:
    scores = np.zeros(num_folds)
    # Perform cross-validation and calculate the average score
    for i in range(num_folds):
        # Get testing set    
        testx = X_train_new[:, i*folder_size:(i+1)*folder_size]
        testy = y_train_new[i*folder_size:(i+1)*folder_size]
        # Get training set    
        trainx = np.hstack((X_train_new[:, 0:(i)*folder_size], X_train_new[:, (i+1)*folder_size:size]))
        trainy = np.hstack((y_train_new[0:(i)*folder_size], y_train_new[(i+1)*folder_size:size]))
        method = mdwsvm(trainx, trainy, w1, c)
        
        pred_y = method.predict(testx)
        score = 1 - within_class_error(testy, pred_y)
        scores[i] = score
        
    # Check if the current value of c is the best so far
    avg_score = np.mean(scores)
    if avg_score > best_score:
        best_c = c
        best_score = avg_score

In [ ]:
print('The best c is', best_c)
print('The best score is',1-best_score)

## Test Error for MDWSVM

1. Use the best C to train the model again using training data
2. Evaluate the model on the test data.

In [ ]:
model1 = mdwsvm(X_train, y_train, w1, best_c)
y_pred = model1.predict(X_test)

In [ ]:
print('The error is', within_class_error(y_test, y_pred))

# Hybrid

## Grid Search for Hybrid Method

Find hyperparameters for hybrid: v and sigma2. 
1. Use C find in MDWSVM. 
2. For each pair of v and sigma2, train the model use training data and evaluate it on the validation data. Return the validation error. 
3. Choose the best pair of hyperparameters that has lowest validation error.
4. Record the running time of the whole training process for each pair of v and sigma2. (Use later to compare the speed of this method and our method)

Range:
1. v in [0.1, 0.9] step = 0.1
2. sigma2 in [1,5,10,15,20,25,30,40,50]

In [ ]:
v_values = np.linspace(0.1,0.9,9)
sigma2_values = [1,5,10,15,20,25,30,40,50]

best_v = 0
best_sigma2 = 0
best_score = -1
time_record_1 = []
for v in v_values:
    for s in sigma2_values:
        print('-'*5+'Start: v=%.2f'%v+' sigma2=%d'%s+'-'*5)
        k = partial(Gaussian_kernel, sigma2=s)
        st = time.time()
        y_pred = hybrid(X_train, y_train, X_val, v, w1, best_c, k)
        et = time.time()
        time_record_1.append(et - st)
        score = 1 - within_class_error(y_val, y_pred)
        if score > best_score:
            best_v = v
            best_sigma2 = s
            best_score = score


In [ ]:
print('The best v is', best_v)
print('The best sigma2 is', best_sigma2)
print('The best score is', best_score)

## Test error for Hybrid Model

1. Use the best v, sigma2 and C to train the model using training data
2. Evaluate the model on the test data

In [ ]:
best_k = partial(Gaussian_kernel, sigma2=best_sigma2)
model2 = hybrid(X_train, y_train, X_test, best_v, w1, best_c, best_k)

In [ ]:
print('The error is', within_class_error(y_test, model2))

# MDWSVM_AD

## Grid Search for MDWSVM_AD Method

Find best hyperparameters
1. For each pair of v, sigma2 and C, train the model use training data and evaluate it on the validation data. Return the validation error. 
2. Choose the best pair of hyperparameters that has lowest validation error.
3. Record the running time of the whole training process for each pair of v, sigma2 and C. (This method should be faster than hybrid and should have a little bit higher accuracy when using the best hyperparameters`)

Range:
1. v in [0.6~0.95] step=0.05
2. sigma in [1, 5] + [10~20] with step=0.5
3. C in [1,5,8,9,10,11]

In [ ]:
v_values = np.arange(0.5,1,0.5)
sigma2_values = [1,5]+np.arange(10,20,1).tolist()
C_values = [1,5,8,9,10,11]
w2 = vertices(5)

best_v_2 = 0
best_sigma2_2 = 0
best_c_2 = 0
best_score_2 = -1
time_record_2 = []
for v in v_values:
    for s in sigma2_values:
        for c in C_values:
            print('-'*5+'Start: v=%.2f'%v+' sigma2=%d'%s+' c=%d'%c+'-'*5)
            k = partial(Gaussian_kernel, sigma2=s)
            st = time.time()
            method = mdwsvm_ad(X_train, y_train, w2, best_c, v, k)
            y_pred = method.predict(X_val)
            et = time.time()
            time_record_2.append(et - st)
            score = 1 - within_class_error(y_val, y_pred)
            if score > best_score_2:
                best_v_2 = v
                best_sigma2_2 = s
                best_c_2 = c
                best_score_2 = score


In [ ]:
print('The best v is', best_v_2)
print('The best sigma2 is', best_sigma2_2)
print('The best C is', best_c_2)
print('The best score is', best_score_2)

## Test Error for MDWSVM_AD Method

Use the best v, sigma2, and C to train the model on the training data and evaluate it on the test data.

In [ ]:
best_k_2 = partial(Gaussian_kernel, sigma2=best_sigma2_2)
model3 = mdwsvm_ad(X_train, y_train, w2, best_c_2, best_v_2, best_k_2)
y_pred = model3.predict(X_test)

In [ ]:
print('The error is', within_class_error(y_test, y_pred))

# For each method, the evaluation on the test data should return two tables.

1. one is about for each digit and letter, the percentage that it is assigned to each class label (5-8 and letter class) (like the table in CS++paper)
2. one is a table for all methods: two rows: one row is what percentage of all the digits are correctly classified, one row is what percentage of all the letters are correctly classified.

# Return a single plot comparing the distribution of running time of hybrid and MDWSVM_AD (can use box plot)

# Don't specifically talk about how we choose hyperparameters in our report, because we don't know anomaly data in practice and don't have the validation set